#### Reference Notebook - https://www.kaggle.com/ravishah1/cots-faster-rcnn-training-w-tf-2-0-od-api-0-474

In [ ]:
import contextlib2
import io
import IPython
import json
import numpy as np
import os
import pathlib
import pandas as pd
import sys
import tensorflow as tf
import time

from PIL import Image, ImageDraw

### Install Object Detectin API

In [ ]:
!git clone https://github.com/tensorflow/models
    
# Check out a certain commit to ensure that future changes in the TF ODT API codebase won't affect this notebook.
!cd models && git checkout ac8d06519

In [ ]:
%%bash
cd models/research

# Compile protos.
protoc object_detection/protos/*.proto --python_out=.

# Install TensorFlow Object Detection API.
# Note: I fixed the version of some dependencies to make it work on Kaggle notebook. In particular:
# * scipy==1.6.3 to avoid the missing GLIBCXX_3.4.26 error
# * tensorflow to 2.6.0 to make it compatible with the CUDA version preinstalled on Kaggle.
# When Kaggle notebook upgrade to TF 2.7, you can use the default setup.py script:
# cp object_detection/packages/tf2/setup.py .
wget https://storage.googleapis.com/odml-dataset/others/setup.py
pip install -q --user .

# Test if the Object Dectection API is working correctly
python object_detection/builders/model_builder_tf2_test.py

In [ ]:
print(tf.__version__)
print(tf.test.is_gpu_available())
print(tf.config.list_physical_devices('GPU'))

### Stratified KFold and Create Dataset

In [ ]:
%%writefile cross_validation_setup.py
"""
Splits a dataframe to a specified cross_validation framework
"""

import pandas as pd
import numpy as np
import argparse
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold

parser = argparse.ArgumentParser(
    description="Cross Validation Setup")
parser.add_argument("-in",
                    "--input_dir",
                    help="The filepath to the input csv.", type=str)
parser.add_argument("-out",
                    "--output_dir",
                    help="The filepath to the output csv.", type=str)
parser.add_argument("-type",
                    "--type",
                    help="The type of cross_validation to use.", type=str)
parser.add_argument("-on",
                    "--on",
                    help="The column to stratisfy on.", type=str)
parser.add_argument("-folds",
                    "--folds",
                    help="The number of folds to create.", type=int)
parser.add_argument("-hold",
                    "--holdout",
                    help="The fold to holdout.", type=int)

args = parser.parse_args()

def cross_validation():
    df = pd.read_csv(args.input_dir)
    df = df[df.annotations!='[]']
    df = df.reset_index(drop=True)
    
    if args.type=='kfold':
        kf = KFold(n_splits=args.folds, shuffle=True, random_state=42)
        for f, (t_, v_) in enumerate(kf.split(X=df)):
            df.loc[v_, 'fold'] = f
            
    elif args.type=='skfold' and args.on=='video_id':
        # uses skf to get even distrubution of data from different videos
        kf = StratifiedKFold(n_splits=args.folds, shuffle=True, random_state=42)
        for f, (t_, v_) in enumerate(kf.split(X=df, y=df.video_id)): 
            df.loc[v_, 'fold'] = f
            
    elif args.type=='gkfold' and args.on=='sequence':
        kf = GroupKFold(n_splits=args.folds)
        for f, (t_, v_) in enumerate(kf.split(X=df, y=df.video_id, groups=df.sequence)): 
            df.loc[v_, 'fold'] = f
            
    else:
        raise Exception('Not Implemented')
        
    df.to_csv(args.output_dir, index=False)
    
if __name__ == '__main__':
    cross_validation()

In [ ]:
%%writefile generate_tfrecords.py

import os
from os.path import exists
import glob
import pandas as pd
import io
import json
import xml.etree.ElementTree as ET
import contextlib2
import argparse

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import tensorflow.compat.v1 as tf
from PIL import Image
from object_detection.utils import dataset_util, label_map_util
from object_detection.dataset_tools import tf_record_creation_util
from collections import namedtuple

# Initiate argument parser
parser = argparse.ArgumentParser(
    description="TensorFlow TFRecord Generator")
parser.add_argument("-c",
                    "--csv_path",
                    help="Path to the train.csv file.", type=str)
parser.add_argument("-o",
                    "--output_path",
                    help="Path of output TFRecord (.record) file.", type=str)
parser.add_argument("-i",
                    "--image_dir",
                    help="Path to the folder where the input image files are stored.", type=str)
parser.add_argument("-t",
                    "--train",
                    help="True if this is a training dataset, false if it is a validation dataset.", type=str)
parser.add_argument("-s",
                    "--shards",
                    help="The number of shards for the dataset", type=int)
parser.add_argument("-f",
                    "--holdout_fold",
                    help="The fold to holdout.", type=int)

args = parser.parse_args()

def create_tf_example(data_df: pd.DataFrame, video_id: int, video_frame: int):
    """
    Create a tf.Example entry for a given training image.
    """
    full_path = os.path.join(args.image_dir, os.path.join(f'video_{video_id}', f'{video_frame}.jpg'))
    with tf.io.gfile.GFile(full_path, 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    if image.format != 'JPEG':
        raise ValueError('Image format not JPEG')

    height = image.size[1] # Image height
    width = image.size[0] # Image width
    filename = f'{video_id}:{video_frame}'.encode('utf8') # Unique id of the image.
    encoded_image_data = None # Encoded image bytes
    image_format = 'jpeg'.encode('utf8') # b'jpeg' or b'png'

    xmins = [] # List of normalized left x coordinates in bounding box (1 per box)
    xmaxs = [] # List of normalized right x coordinates in bounding box
             # (1 per box)
    ymins = [] # List of normalized top y coordinates in bounding box (1 per box)
    ymaxs = [] # List of normalized bottom y coordinates in bounding box
             # (1 per box)
    classes_text = [] # List of string class name of bounding box (1 per box)
    classes = [] # List of integer class id of bounding box (1 per box)

    rows = data_df[(data_df.video_id == video_id) & (data_df.video_frame == video_frame)]
    for _, row in rows.iterrows():
        annotations = json.loads(row.annotations.replace("'", '"'))
        for annotation in annotations:
            xmins.append(annotation['x'] / width) 
            xmaxs.append((annotation['x'] + annotation['width']) / width) 
            ymins.append(annotation['y'] / height) 
            ymaxs.append((annotation['y'] + annotation['height']) / height) 

            classes_text.append('COTS'.encode('utf8'))
            classes.append(1)

    tf_example = tf.train.Example(features=tf.train.Features(feature={
      'image/height': dataset_util.int64_feature(height),
      'image/width': dataset_util.int64_feature(width),
      'image/filename': dataset_util.bytes_feature(filename),
      'image/source_id': dataset_util.bytes_feature(filename),
      'image/encoded': dataset_util.bytes_feature(encoded_jpg),
      'image/format': dataset_util.bytes_feature(image_format),
      'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
      'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
      'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
      'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
      'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
      'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))

    return tf_example

def create_labels_file():
    label_map_str = """
    item {
        id: 1
        name: 'COTS'
        }
                    """

    if exists('dataset/label_map.pbtxt') is False:
        with open('dataset/label_map.pbtxt', 'w') as f:
            f.write(label_map_str)
        print('Successfully created label_map.pbtxt file')

if __name__ == '__main__':

    # label file
    create_labels_file()
    #writer = tf.python_io.TFRecordWriter(args.output_path)
    
    # setup df
    data_df = pd.read_csv(args.csv_path)
    if args.train =='train':
        data_df = data_df[data_df.fold != args.holdout_fold].reset_index(drop=True)
    else:
        data_df = data_df[data_df.fold == args.holdout_fold].reset_index(drop=True)
    
    # make records
    with contextlib2.ExitStack() as tf_record_close_stack:
        output_tfrecords = tf_record_creation_util.open_sharded_output_tfrecords(
            tf_record_close_stack, args.output_path, args.shards)
        
        for index, row in data_df.iterrows():
            tf_example = create_tf_example(data_df, row.video_id, row.video_frame)
            output_shard_index = index % args.shards
            output_tfrecords[output_shard_index].write(tf_example.SerializeToString())
    #writer.close()
    print('Successfully created the TFRecord file: {}'.format(args.output_path))

In [ ]:
%%bash

python cross_validation_setup.py \
    -in ../input/tensorflow-great-barrier-reef/train.csv \
    -out train.csv \
    -type skfold \
    -on video_id \
    -folds 10 \
    -hold 0

mkdir dataset

python generate_tfrecords.py \
    -c train.csv \
    -o dataset/cots_train \
    -i ../input/tensorflow-great-barrier-reef/train_images \
    -t train \
    -s 4 \
    -f 0

python generate_tfrecords.py \
    -c train.csv \
    -o dataset/cots_val \
    -i ../input/tensorflow-great-barrier-reef/train_images \
    -t valid \
    -s 4 \
    -f 0

### Download Pretrained Model

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz
!tar -xvzf faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz

In [ ]:
!rm faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz

### Edit Config

In [ ]:
from string import Template

config_file_template = """
# Faster R-CNN with Resnet-50 (v1)
# Trained on COCO, initialized from Imagenet classification checkpoint
#
# Train on TPU-8
#
# Achieves 31.8 mAP on COCO17 val

model {
  faster_rcnn {
    num_classes: 1
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 960
        max_dimension: 960
        pad_to_max_dimension: true
      }
    }
    feature_extractor {
      type: 'faster_rcnn_resnet101_keras'
      batch_norm_trainable: true
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.01
        }
      }
    }
    first_stage_nms_score_threshold: 0.0
    first_stage_nms_iou_threshold: 0.7
    first_stage_max_proposals: 300
    first_stage_localization_loss_weight: 2.0
    first_stage_objectness_loss_weight: 1.0
    initial_crop_size: 14
    maxpool_kernel_size: 2
    maxpool_stride: 2
    second_stage_box_predictor {
      mask_rcnn_box_predictor {
        use_dropout: false
        dropout_keep_probability: 1.0
        fc_hyperparams {
          op: FC
          regularizer {
            l2_regularizer {
              weight: 0.0
            }
          }
          initializer {
            variance_scaling_initializer {
              factor: 1.0
              uniform: true
              mode: FAN_AVG
            }
          }
        }
        share_box_across_classes: true
      }
    }
    second_stage_post_processing {
      batch_non_max_suppression {
        score_threshold: 0.0
        iou_threshold: 0.6
        max_detections_per_class: 100
        max_total_detections: 300
      }
      score_converter: SOFTMAX
    }
    second_stage_localization_loss_weight: 2.0
    second_stage_classification_loss_weight: 1.0
    use_static_shapes: true
    use_matmul_crop_and_resize: true
    clip_anchors_to_image: true
    use_static_balanced_label_sampler: true
    use_matmul_gather_in_matcher: true
  }
}

train_config: {
  fine_tune_checkpoint: "faster_rcnn_resnet101_v1_640x640_coco17_tpu-8/checkpoint/ckpt-0"
  fine_tune_checkpoint_version: V2
  fine_tune_checkpoint_type: "detection"
  batch_size: 1
  sync_replicas: false
  startup_delay_steps: 0
  replicas_to_aggregate: 1
  use_bfloat16: false
  num_steps: $training_steps
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
  data_augmentation_options {
    random_scale_crop_and_pad_to_square {
      output_size: 1280
      scale_min: 0.5
      scale_max: 2.0
    }
  }
  optimizer {
    momentum_optimizer: {
      learning_rate: {
        cosine_decay_learning_rate {
          learning_rate_base: 5e-3
          total_steps: $training_steps
          warmup_learning_rate: 5e-4
          warmup_steps: $warmup_steps
        }
      }
      momentum_optimizer_value: 0.9
    }
    use_moving_average: false
  }
  max_number_of_boxes: 100
  unpad_groundtruth_tensors: false
}

train_input_reader: {
  label_map_path: "dataset/label_map.pbtxt"
  tf_record_input_reader {
    input_path: "dataset/cots_train-?????-of-00004"
  }
}

eval_config: {
  metrics_set: "coco_detection_metrics"
  use_moving_averages: false
  batch_size: 2;
}

eval_input_reader: {
  label_map_path: "dataset/label_map.pbtxt"
  shuffle: false
  num_epochs: 1
  tf_record_input_reader {
    input_path: "dataset/cots_val-?????-of-00004"
  }
}
"""

In [ ]:
# Define the training pipeline

TRAINING_STEPS = 1800
WARMUP_STEPS = 200
PIPELINE_CONFIG_PATH='dataset/pipeline.config'

pipeline = Template(config_file_template).substitute(
    training_steps=TRAINING_STEPS, warmup_steps=WARMUP_STEPS)

with open(PIPELINE_CONFIG_PATH, 'w') as f:
    f.write(pipeline)

In [ ]:
MODEL_DIR='cots_faster_rcnn_resnet101'
!mkdir {MODEL_DIR}

### Train

In [ ]:
!python models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={PIPELINE_CONFIG_PATH} \
    --model_dir={MODEL_DIR} \
    --alsologtostderr

### Export

In [ ]:
!mkdir {MODEL_DIR}/output

In [ ]:
!python models/research/object_detection/exporter_main_v2.py \
    --input_type image_tensor \
    --pipeline_config_path={PIPELINE_CONFIG_PATH} \
    --trained_checkpoint_dir={MODEL_DIR} \
    --output_directory={MODEL_DIR}/output

In [ ]:
ls {MODEL_DIR}/output

### Clean up

In [ ]:
!rm -rf dataset/
!rm -rf faster_rcnn_resnet101_v1_640x640_coco17_tpu-8/
!rm -rf models/

In [ ]:
# # Remove the dataset files to save space.
# !rm -rf dataset
# !rm -rf train_images
# !rm tensorflow-great-barrier-reef.zip

# # Remove other data downloaded during training.
# !rm -rf models
# !rm faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz